<a href="https://colab.research.google.com/github/William14045/N_G_C/blob/main/GC_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Challenge 3

Nama : William

Batch : RMT-020

# Introduction
## Problem Statement
menganalisa transaksi penjualan pada thelook E-commerse dalam Kuartal 1 sampai Kuartal 3 tahun 2022 dan mencari Trend transaksi untuk kuartal ke 4 tahun 2022
## Objective
mengeksplorasi data yang telah disediakan untuk mendapatkan insight

#Data Exploration

In [1]:
# menghubungkan Google colab dengan BigQuery
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "abcd-388207"
client = bigquery.Client(project=project_id)

Authenticated


In [2]:
# Import library / module yang akan digunakan
import pandas as pd
import numpy as np
pd.set_option("display.precision",2)

In [3]:
# Mencari Jumlah transaksi per bulan
df = client.query('''select
  extract(month from delivered_at) as bulan,
  extract(quarter from delivered_at) as Quarter,
  count(status) as total
from bigquery-public-data.thelook_ecommerce.orders
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
group by bulan,Quarter
order by bulan''').to_dataframe()
df

,bulan,Quarter,total
0,1,1,640
1,2,1,608
2,3,1,716
3,4,2,764
4,5,2,813
5,6,2,860
6,7,3,901
7,8,3,970
8,9,3,1011


dari tabel diatas dapat dilihat meskipun ada terjadi penurunan di setiap kuartal tetapi secara keseluruhan selama Q1 sampai Q3 jumlah transaksi Cenderung meningkat

In [4]:
# mencari Total Pendapatan Per bulan
fd = client.query('''select
extract(quarter from delivered_at) as Quarter,
extract(month from delivered_at) as bulan,
sum(sale_price) as total
from bigquery-public-data.thelook_ecommerce.order_items
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
group by 1,2
order by 1,2''').to_dataframe()
fd

,Quarter,bulan,total
0,1,1,61358.90
1,1,2,54659.43
2,1,3,61551.60
3,2,4,65090.61
4,2,5,68936.77
5,2,6,79398.42
6,3,7,79438.88
7,3,8,81395.75
8,3,9,85902.44


berdasarkan data yang didapatkan bahwa total penjualan tertinggi ada pada bulan terakhir kuartal 3 yaitu bulan september sedangkan penjualan terendah berada pada bulan februari

In [5]:
# Mencari Jumlah user yang melakukan transaksi Per bulan
dff = client.query('''SELECT extract(month from delivered_at) AS month, COUNT(DISTINCT user_id) AS distinct_users
from bigquery-public-data.thelook_ecommerce.orders
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
GROUP BY month
ORDER BY month
''').to_dataframe()
dff

,month,distinct_users
0,1,633
1,2,604
2,3,712
3,4,762
4,5,810
5,6,856
6,7,896
7,8,962
8,9,1009


sama seperti 2 tabel sebelumnya, jumlah user yang melakukan transaksi juga memiliki pola Kenaikan dari awal tahun sampai dengan Kuartal 3

In [6]:
# Mencari kategori Produk paling banyak dibeli per bulan

ac = client.query('''select c.bulan,c.total, b.kategori
from( select t.bulan,max(t.jumlah) as total
from (SELECT extract(MONTH from ord.delivered_at) AS bulan, prod.category as kategori, COUNT(*) AS jumlah
FROM bigquery-public-data.thelook_ecommerce.order_items as ord
left join bigquery-public-data.thelook_ecommerce.products as prod
on ord.product_id = prod.id
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
GROUP BY bulan, kategori
ORDER BY bulan, jumlah DESC) as t
group by bulan
order by bulan) as c
left join(SELECT extract(MONTH from ord.delivered_at) AS bulan, prod.category as kategori, COUNT(*) AS jumlah
FROM bigquery-public-data.thelook_ecommerce.order_items as ord
left join bigquery-public-data.thelook_ecommerce.products as prod
on ord.product_id = prod.id
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
GROUP BY bulan, kategori
ORDER BY bulan, jumlah DESC) as b
on c.total = b.jumlah
order by c.bulan''').to_dataframe()
ac

,bulan,total,kategori
0,1,69,Intimates
1,1,69,Swim
2,1,69,Outerwear & Coats
3,2,77,Accessories
4,2,77,Active
5,2,77,Tops & Tees
6,3,75,Intimates
7,4,88,Intimates
8,4,88,Shorts
9,4,88,Jeans


Berdasarkan Jumlah transaksi didapat bahwa Kategori Intimates dan Top&Tees merupakan favorit dari User selama 3 Kuartal

Intimates : Masuk Dalam kategori paling banyak dibeli selama 7 bulan

Top & Tees : Masuk Dalam kategori paling banyak dibeli selama 4 bulan


In [7]:
# Mencari Kategori produk yang menghasilkan Pendapatan terbanyak per bulan
dc = client.query('''select c.bulan, b.kategori, c.total
from(SELECT bulan, max(total) as total
from(SELECT extract(MONTH from ord.delivered_at) AS bulan, prod.category as kategori, sum(ord.sale_price) AS total
FROM bigquery-public-data.thelook_ecommerce.order_items as ord
left join bigquery-public-data.thelook_ecommerce.products as prod
on ord.product_id = prod.id
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
group by 1,2
order by 1)
group by 1) c
left join(SELECT extract(MONTH from ord.delivered_at) AS bulan, prod.category as kategori, sum(ord.sale_price) AS total
FROM bigquery-public-data.thelook_ecommerce.order_items as ord
left join bigquery-public-data.thelook_ecommerce.products as prod
on ord.product_id = prod.id
where status = 'Complete' and  delivered_at >= '2022-1-1' and delivered_at <= '2022-9-30'
group by 1,2
order by 1) b
on c.total = b.total
order by 1''').to_dataframe()
dc

,bulan,kategori,total
0,1,Outerwear & Coats,9256.59
1,2,Outerwear & Coats,8612.03
2,3,Outerwear & Coats,6852.92
3,4,Outerwear & Coats,8868.73
4,5,Outerwear & Coats,8792.34
5,6,Outerwear & Coats,10127.70
6,7,Jeans,10037.76
7,8,Outerwear & Coats,8071.20
8,9,Outerwear & Coats,9348.54


meskipun tidak dibeli sebanyak kategori Intimates dan Top&Tees, kategori Outerwear & Coats menghasilkan uang paling banyak hampir setiap bulannya

In [20]:
d = {'User': dff['distinct_users'], 'Transaksi': df['total'], 'Penjualan' : fd['total'] } # membuat Tabel baru untuk dicari nilai korelasi nya
kor = pd.DataFrame(d)
kor

,User,Transaksi,Penjualan
0,633,640,61358.90
1,604,608,54659.43
2,712,716,61551.60
3,762,764,65090.61
4,810,813,68936.77
5,856,860,79398.42
6,896,901,79438.88
7,962,970,81395.75
8,1009,1011,85902.44


In [15]:
kor.corr() # mencari nilai korelasi

,User,Transaksi,Penjualan
User,1.00,1.00,0.97
Transaksi,1.00,1.00,0.97
Penjualan,0.97,0.97,1.00


berdasarkan nilai korelasi yang didapat, jumlah transaksi lebih berkaitan dengan jumlah user. walaupun begitu, total penjualan juga memiliki keterkaitan yang besar

# POIN PERTANYAAN

•Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

    trend transaksi dapat diukur kemungkinannya berdasarkan data-data yang tersedia pada bulan sebelumnya

•Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

    jika target penjualan selama kuartal 4 ialah $250.000 maka target mungkin dapat dicapai, tetapi jika hanya pada awal kuartal (1-2 bulan) maka target tersebut tidak masuk akal berdasarkan data-data dari bulan sebelumnya yang dimana pendapatan per bulan berkisar antara $60.000 - $85.000

•CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!



In [24]:
kor['persentase (%)'] = ((kor['Penjualan'] - kor['Penjualan'].shift(1)) / kor['Penjualan'].shift(1)) * 100 # menambahkan kolom baru
kor

,User,Transaksi,Penjualan,persentase (%)
0,633,640,61358.90,NaN
1,604,608,54659.43,-10.92
2,712,716,61551.60,12.61
3,762,764,65090.61,5.75
4,810,813,68936.77,5.91
5,856,860,79398.42,15.18
6,896,901,79438.88,0.05
7,962,970,81395.75,2.46
8,1009,1011,85902.44,5.54


kolom persentase disini dapat mewakilkan banyaknya user, jumlah transaksi dan total penjualan selama 3 kuartal terakhir karena adanya korelasi antara ketiga kolom tersebut. persentase bernilai positif menunjukkan bahwa dalam 3 kuartal terakhir trend transaksi mengalami kenaikan. hal tersebut berkaitan dengan GMV nya yang juga ikut naik per kuartal nya.

    jadi kesimpulannya perusahaan layak mendapatkan investor baru dikarenakan Trend transaksi yang positif selama 3 kuartal terakhir

# Summary
meskipun adanya penurunan transaksi pada awal kuartal 1 tahun 2022, secara keseluruhan Q1-Q3 2022 Thelook E-commerse mengalami pertumbuhan baik dari segi banyaknya user yang bertransaksi, jumlah transaksi, serta jumlah penjualannya